In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
import art
import asyncio
from dotenv import load_dotenv
import json
import openai
import random
import re
from typing import TypedDict

load_dotenv()


class TemporalCluePuzzle(TypedDict):
    num_clues: int
    prompt: str
    solution: dict[str, str]


puzzles: list[TemporalCluePuzzle] = json.load(open("./data/temporal-clue/puzzles.json"))
val_puzzles = puzzles[:64]
test_puzzles = puzzles[64:128]
train_puzzles = puzzles[128:]
random.seed(42)
random.shuffle(train_puzzles)


api = art.LocalAPI(wandb_project="agent-reinforcement-training")
model = await api._get_or_create_model(
    name="temporal-clue-unsloth-002",
    base_model="Qwen/Qwen2.5-14B-Instruct",
    _config={"init_args": {"gpu_memory_utilization": 0.775}},
)


async def rollout(
    client: openai.AsyncOpenAI, puzzle: TemporalCluePuzzle
) -> art.Trajectory:
    messages: art.Messages = [{"role": "user", "content": puzzle["prompt"]}]
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    num_correct = 0
    for key, value in puzzle["solution"].items():
        if matches := re.findall(rf"{key}\. ([A-Za-z \.:-]+)", content):
            match = matches[-1]
            if match.strip().lower() == value.lower():
                num_correct += 1
    reward = acc = num_correct / len(puzzle["solution"])
    return art.Trajectory(
        messages_and_choices=[*messages, choice], reward=reward, metrics={"acc": acc}
    )


stride = 4
openai_client = await model.openai_client()
for i in range(await model.get_step(), 1_000):
    val_groups, train_groups = await asyncio.gather(
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(rollout(openai_client, puzzle) for _ in range(2))
                for puzzle in val_puzzles
            ),
            pbar_desc="val",
        ),
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(rollout(openai_client, puzzle) for _ in range(50))
                for puzzle in train_puzzles[i * stride : (i + 1) * stride]
            ),
            pbar_desc="train",
        ),
    )
    await model.log(val_groups)
    await model.delete_checkpoints()
    await model.train(
        train_groups,
        config=art.TrainConfig(learning_rate=5e-5),
    )

/home/gcpuser/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-10 22:04:14 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-14b-instruct-unsloth-bnb-4bit with actual GPU utilization = 76.81%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.11 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. 

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.23it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.78it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.58it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.14it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.49it/s]



INFO 04-10 22:04:57 model_runner.py:1115] Loading model weights took 10.6011 GB
INFO 04-10 22:04:57 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-10 22:05:00 worker.py:267] Memory profiling takes 3.23 seconds
INFO 04-10 22:05:00 worker.py:267] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.77) = 60.77GiB
INFO 04-10 22:05:00 worker.py:267] model weights take 10.60GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 4.26GiB; the rest of the memory reserved for KV Cache is 45.76GiB.
INFO 04-10 22:05:01 executor_base.py:111] # cuda blocks: 15617, # CPU blocks: 2048
INFO 04-10 22:05:01 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 7.63x
INFO 04-10 22:05:03 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory erro

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:41<00:00,  1.19it/s]


...

wandb: WARNING Tried to log to step 1 that is less than the current step 2094. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


No "val/reward" metric found in history
Deleted checkpoint ./.art/models/temporal-clue-unsloth-002/0000
Packed 200 trajectories into 32 sequences of length 8192


train:   0%|          | 0/32 [00:00<?, ?it/s]

wandb: WARNING Tried to log to step 1 that is less than the current step 2094. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 1 that is less than the current step 2094. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


CancelledError: 